# AI-Powered Code Optimization Framework

A comprehensive framework that uses Groq's language models to analyze and optimize Python code for:
- **Performance** - Speed and algorithmic efficiency
- **Readability** - Code clarity and maintainability
- **Memory** - Memory usage optimization
- **General** - Overall code quality improvements

## Table of Contents
1. [Setup and Dependencies](#setup)
2. [Core Data Structures](#data-structures)
3. [Code Analysis Engine](#code-analyzer)
4. [Optimization Agents](#optimization-agents)
5. [Code Validation](#code-validator)
6. [Main Optimization Framework](#main-framework)
7. [Usage Examples](#usage-examples)
8. [Advanced Features](#advanced-features)

## Setup and Dependencies {#setup}

In [ ]:
# Install required packages
!pip install groq nest-asyncio

import ast
import re
import time
import json
import logging
from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass, asdict
from enum import Enum
from abc import ABC, abstractmethod
import asyncio
from datetime import datetime

try:
    from groq import Groq
    print("✅ Groq imported successfully")
except ImportError:
    print("❌ Please install groq: pip install groq")
    raise

# For Jupyter/Colab environments
try:
    import nest_asyncio
    nest_asyncio.apply()
    NEST_ASYNCIO_AVAILABLE = True
    print("✅ nest_asyncio configured for Jupyter")
except ImportError:
    NEST_ASYNCIO_AVAILABLE = False
    print("⚠️ nest_asyncio not available (install with: pip install nest_asyncio)")

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("✅ All imports loaded successfully!")

In [ ]:
# Configuration - Set your Groq API key here
GROQ_API_KEY = "your-groq-api-key-here"  # Replace with your actual API key
MODEL_NAME = "meta-llama/llama-4-maverick-17b-128e-instruct"

if GROQ_API_KEY == "your-groq-api-key-here":
    print("⚠️ Please set your GROQ_API_KEY in the cell above")
    print("   You can get one from: https://console.groq.com/keys")
else:
    print("✅ API key configured successfully!")
    print(f"📊 Using model: {MODEL_NAME}")

## Core Data Structures {#data-structures}

Define the core data structures for optimization requests, modifications, and results.

In [ ]:
class OptimizationType(Enum):
    PERFORMANCE = "performance"
    READABILITY = "readability"
    MEMORY = "memory"
    MAINTAINABILITY = "maintainability"
    SECURITY = "security"
    GENERAL = "general"

print("✅ OptimizationType enum defined")
print(f"Available optimization types: {[opt.value for opt in OptimizationType]}")

In [ ]:
@dataclass
class OptimizationRequest:
    code: str
    query: str
    optimization_type: OptimizationType
    constraints: Optional[Dict[str, Any]] = None
    max_iterations: int = 3

@dataclass
class Modification:
    type: str  # e.g., "performance", "style", "logic"
    description: str
    line_numbers: Optional[List[int]] = None
    before: Optional[str] = None
    after: Optional[str] = None
    impact: Optional[str] = None  # e.g., "high", "medium", "low"

print("✅ OptimizationRequest and Modification dataclasses defined")

In [ ]:
@dataclass
class OptimizationResult:
    original_code: str
    optimized_code: str
    modifications: List[Modification]
    analysis: Dict[str, Any]
    confidence_score: float
    iteration_count: int
    optimization_type: str
    timestamp: str

    def to_json(self, pretty: bool = True) -> str:
        """Convert the result to JSON format."""
        result_dict = {
            "optimization_summary": {
                "type": self.optimization_type,
                "confidence_score": round(self.confidence_score, 3),
                "iteration_count": self.iteration_count,
                "timestamp": self.timestamp,
                "total_modifications": len(self.modifications)
            },
            "modifications": [
                {
                    "type": mod.type,
                    "description": mod.description,
                    "line_numbers": mod.line_numbers,
                    "before": mod.before,
                    "after": mod.after,
                    "impact": mod.impact
                } for mod in self.modifications
            ],
            "code": {
                "original": self.original_code,
                "optimized": self.optimized_code,
                "lines_changed": self._calculate_lines_changed(),
                "size_change": {
                    "original_lines": len(self.original_code.split('\n')),
                    "optimized_lines": len(self.optimized_code.split('\n')),
                    "difference": len(self.optimized_code.split('\n')) - len(self.original_code.split('\n'))
                }
            },
            "analysis": self.analysis
        }

        if pretty:
            return json.dumps(result_dict, indent=2, ensure_ascii=False)
        return json.dumps(result_dict, ensure_ascii=False)

    def _calculate_lines_changed(self) -> int:
        """Calculate approximate number of lines changed."""
        original_lines = set(self.original_code.split('\n'))
        optimized_lines = set(self.optimized_code.split('\n'))
        return len(original_lines.symmetric_difference(optimized_lines))

    def display_summary(self):
        """Display a formatted summary of the optimization results."""
        print(f"🎯 Optimization Type: {self.optimization_type}")
        print(f"📊 Confidence Score: {self.confidence_score:.2%}")
        print(f"🔄 Iterations: {self.iteration_count}")
        print(f"📝 Modifications: {len(self.modifications)}")
        print(f"📏 Lines changed: {self._calculate_lines_changed()}")
        
        if self.modifications:
            print("\n🔧 Key Improvements:")
            for i, mod in enumerate(self.modifications[:3], 1):  # Show top 3
                print(f"   {i}. [{mod.type}] {mod.description}")

print("✅ OptimizationResult dataclass defined with JSON export and display capabilities")

## Code Analysis Engine {#code-analyzer}

Analyzes code complexity, detects code smells, and provides metrics.

In [ ]:
class CodeAnalyzer:
    """Analyzes code structure, complexity, and potential issues."""

    @staticmethod
    def analyze_complexity(code: str) -> Dict[str, Any]:
        """Calculate cyclomatic complexity and other metrics."""
        try:
            tree = ast.parse(code)

            complexity = 1  # Base complexity
            lines = len(code.split('\n'))
            functions = 0
            classes = 0

            for node in ast.walk(tree):
                if isinstance(node, (ast.If, ast.While, ast.For, ast.AsyncFor)):
                    complexity += 1
                elif isinstance(node, ast.FunctionDef):
                    functions += 1
                elif isinstance(node, ast.ClassDef):
                    classes += 1

            return {
                "cyclomatic_complexity": complexity,
                "lines_of_code": lines,
                "function_count": functions,
                "class_count": classes,
                "complexity_per_line": complexity / max(lines, 1)
            }
        except SyntaxError:
            return {"error": "Invalid Python syntax"}

    @staticmethod
    def find_code_smells(code: str) -> List[Dict[str, Any]]:
        """Identify common code smells and anti-patterns with detailed info."""
        smells = []

        # Check for overly long lines
        for i, line in enumerate(code.split('\n'), 1):
            if len(line) > 100:
                smells.append({
                    "type": "style",
                    "issue": "long_line",
                    "line": i,
                    "description": f"Line {i}: Overly long line ({len(line)} chars)",
                    "severity": "medium"
                })

        # Check for nested loops
        if re.search(r'for.*:\s*\n.*for.*:', code, re.MULTILINE):
            smells.append({
                "type": "performance",
                "issue": "nested_loops",
                "description": "Nested loops detected - consider optimization",
                "severity": "high"
            })

        # Check for global variables
        global_matches = list(re.finditer(r'^global\s+(\w+)', code, re.MULTILINE))
        for match in global_matches:
            line_num = code[:match.start()].count('\n') + 1
            smells.append({
                "type": "maintainability",
                "issue": "global_variable",
                "line": line_num,
                "description": f"Global variable '{match.group(1)}' found - consider encapsulation",
                "severity": "medium"
            })

        # Check for magic numbers
        magic_numbers = re.findall(r'\b\d{2,}\b', code)
        if magic_numbers:
            smells.append({
                "type": "maintainability",
                "issue": "magic_numbers",
                "description": f"Magic numbers found: {set(magic_numbers)}",
                "severity": "low"
            })

        return smells

    def analyze_and_display(self, code: str, title: str = "Code Analysis"):
        """Analyze code and display results in a readable format."""
        complexity = self.analyze_complexity(code)
        smells = self.find_code_smells(code)
        
        print(f"📊 {title}")
        print("=" * (len(title) + 4))
        
        if "error" in complexity:
            print(f"❌ {complexity['error']}")
            return
        
        print(f"🔢 Cyclomatic Complexity: {complexity['cyclomatic_complexity']}")
        print(f"📏 Lines of Code: {complexity['lines_of_code']}")
        print(f"🔧 Functions: {complexity['function_count']}")
        print(f"🏗️ Classes: {complexity['class_count']}")
        print(f"📈 Complexity/Line: {complexity['complexity_per_line']:.3f}")
        
        if smells:
            print(f"\n🚨 Code Smells Found: {len(smells)}")
            for i, smell in enumerate(smells[:5], 1):  # Show first 5
                severity_emoji = {"high": "🔴", "medium": "🟡", "low": "🟢"}
                emoji = severity_emoji.get(smell['severity'], "⚪")
                print(f"   {i}. {emoji} {smell['description']}")
            if len(smells) > 5:
                print(f"   ... and {len(smells) - 5} more")
        else:
            print("\n✅ No code smells detected")

# Test the analyzer
analyzer = CodeAnalyzer()
test_code = """
def example_function(x, y):
    if x > 0:
        for i in range(100):  # Magic number
            if i > 50:  # Another magic number
                return x + y
    return 0
"""

analyzer.analyze_and_display(test_code, "Sample Code Analysis")

## Optimization Agents {#optimization-agents}

Specialized agents for different types of code optimization.

In [ ]:
class BaseOptimizationAgent(ABC):
    """Base class for optimization agents."""

    def __init__(self, groq_client: Groq, model_name: str = "meta-llama/llama-4-maverick-17b-128e-instruct"):
        self.groq_client = groq_client
        self.model_name = model_name
        self.analyzer = CodeAnalyzer()

    @abstractmethod
    def get_optimization_prompt(self, request: OptimizationRequest) -> str:
        """Generate the optimization prompt for the specific agent type."""
        pass

    async def optimize(self, request: OptimizationRequest) -> Tuple[str, List[Modification]]:
        """Optimize code using Groq model and extract modifications."""
        prompt = self.get_optimization_prompt(request)

        try:
            response = self.groq_client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            content = response.choices[0].message.content

            # Extract code and modifications
            code = self.extract_code_from_response(content)
            modifications = self.extract_modifications_from_response(content, request.code, code)

            return code, modifications
        except Exception as e:
            logger.error(f"Groq API error: {e}")
            raise

    def extract_code_from_response(self, response: str) -> str:
        """Extract Python code from the model response."""
        # Look for code blocks
        code_match = re.search(r'```python\n(.*?)\n```', response, re.DOTALL)
        if code_match:
            return code_match.group(1)

        # Look for code without markdown
        code_match = re.search(r'```\n(.*?)\n```', response, re.DOTALL)
        if code_match:
            return code_match.group(1)

        # If no code blocks found, return the response as is
        return response.strip()

    def extract_modifications_from_response(self, response: str, original_code: str, optimized_code: str) -> List[Modification]:
        """Extract modification details from the response."""
        modifications = []

        # Parse the response for improvement descriptions
        improvements_section = re.search(r'(?:improvements?|changes?|modifications?).*?:(.*?)(?:\n\n|\n[A-Z]|\Z)',
                                       response, re.IGNORECASE | re.DOTALL)

        if improvements_section:
            improvements_text = improvements_section.group(1)

            # Split into individual improvements
            improvement_lines = [line.strip() for line in improvements_text.split('\n')
                               if line.strip() and not line.strip().startswith('```')]

            for improvement in improvement_lines:
                if improvement.startswith(('-', '*', '•', '1.', '2.', '3.')):
                    # Clean up the improvement text
                    clean_improvement = re.sub(r'^[-*•\d.)\s]+', '', improvement).strip()

                    # Categorize the modification
                    mod_type = self._categorize_modification(clean_improvement)
                    impact = self._assess_impact(clean_improvement)

                    modifications.append(Modification(
                        type=mod_type,
                        description=clean_improvement,
                        impact=impact
                    ))

        # If no structured improvements found, create generic ones
        if not modifications:
            modifications.append(Modification(
                type="general",
                description="Code optimized using AI analysis",
                impact="medium"
            ))

        return modifications

    def _categorize_modification(self, description: str) -> str:
        """Categorize modification based on description."""
        desc_lower = description.lower()

        if any(word in desc_lower for word in ['performance', 'speed', 'efficient', 'algorithm', 'complexity']):
            return "performance"
        elif any(word in desc_lower for word in ['readable', 'naming', 'comment', 'structure', 'format']):
            return "readability"
        elif any(word in desc_lower for word in ['memory', 'generator', 'space']):
            return "memory"
        elif any(word in desc_lower for word in ['maintainable', 'global', 'encapsulation']):
            return "maintainability"
        else:
            return "general"

    def _assess_impact(self, description: str) -> str:
        """Assess the impact level of a modification."""
        desc_lower = description.lower()

        if any(word in desc_lower for word in ['significant', 'major', 'dramatic', 'substantial']):
            return "high"
        elif any(word in desc_lower for word in ['minor', 'small', 'slight']):
            return "low"
        else:
            return "medium"

print("✅ BaseOptimizationAgent defined with code extraction and modification detection")

In [ ]:
class PerformanceOptimizationAgent(BaseOptimizationAgent):
    """Agent specialized in performance optimization."""

    def get_optimization_prompt(self, request: OptimizationRequest) -> str:
        analysis = self.analyzer.analyze_complexity(request.code)
        smells = self.analyzer.find_code_smells(request.code)

        return f"""
You are a Python performance optimization expert. Optimize the following code for better performance.

QUERY: {request.query}

ORIGINAL CODE:
{request.code}

CODE ANALYSIS:
- Complexity: {analysis.get('cyclomatic_complexity', 'N/A')}
- Lines of code: {analysis.get('lines_of_code', 'N/A')}
- Code smells: {[smell['description'] for smell in smells]}

OPTIMIZATION FOCUS:
- Reduce time complexity where possible
- Optimize loops and iterations
- Use efficient data structures
- Minimize function calls in hot paths
- Consider algorithmic improvements

Please provide:
1. Optimized Python code (wrapped in ```python code blocks)
2. List of specific improvements made (use bullet points starting with -)
3. Estimated performance gain for each improvement

Only return valid, executable Python code that maintains the original functionality.
"""

print("✅ PerformanceOptimizationAgent defined")

In [ ]:
class ReadabilityOptimizationAgent(BaseOptimizationAgent):
    """Agent specialized in readability optimization."""

    def get_optimization_prompt(self, request: OptimizationRequest) -> str:
        return f"""
You are a Python code readability expert. Improve the following code for better readability and maintainability.

QUERY: {request.query}

ORIGINAL CODE:
{request.code}

OPTIMIZATION FOCUS:
- Improve variable and function names
- Add appropriate comments and docstrings
- Break down complex functions
- Follow PEP 8 conventions
- Improve code structure and organization

Please provide:
1. Optimized Python code (wrapped in ```python code blocks)
2. List of readability improvements made (use bullet points starting with -)

Maintain all original functionality while making the code more readable.
"""

print("✅ ReadabilityOptimizationAgent defined")

In [ ]:
class MemoryOptimizationAgent(BaseOptimizationAgent):
    """Agent specialized in memory optimization."""

    def get_optimization_prompt(self, request: OptimizationRequest) -> str:
        return f"""
You are a Python memory optimization expert. Optimize the following code for better memory usage.

QUERY: {request.query}

ORIGINAL CODE:
{request.code}

OPTIMIZATION FOCUS:
- Reduce memory footprint
- Use generators instead of lists where appropriate
- Optimize data structure choices
- Eliminate memory leaks
- Use memory-efficient algorithms

Please provide:
1. Optimized Python code (wrapped in ```python code blocks)
2. List of memory optimizations made (use bullet points starting with -)

Ensure the code maintains original functionality with reduced memory usage.
"""

print("✅ MemoryOptimizationAgent defined")

In [ ]:
class GeneralOptimizationAgent(BaseOptimizationAgent):
    """General-purpose optimization agent."""

    def get_optimization_prompt(self, request: OptimizationRequest) -> str:
        analysis = self.analyzer.analyze_complexity(request.code)
        smells = self.analyzer.find_code_smells(request.code)

        return f"""
You are a Python optimization expert. Improve the following code based on the specific query.

QUERY: {request.query}

ORIGINAL CODE:
{request.code}

CODE ANALYSIS:
- Complexity: {analysis.get('cyclomatic_complexity', 'N/A')}
- Lines of code: {analysis.get('lines_of_code', 'N/A')}
- Functions: {analysis.get('function_count', 'N/A')}
- Classes: {analysis.get('class_count', 'N/A')}
- Code smells: {[smell['description'] for smell in smells]}

Please analyze the code and the query, then provide optimized code that addresses the specific requirements while improving:
- Performance where possible
- Code readability and maintainability
- Best practices adherence

Provide:
1. Optimized Python code (wrapped in ```python code blocks)
2. List of specific improvements made (use bullet points starting with -)
3. Brief rationale for each optimization

Ensure the optimized code maintains all original functionality.
"""

print("✅ GeneralOptimizationAgent defined")
print("✅ All optimization agents are ready!")

## Code Validation {#code-validator}

Validates optimized code for syntax and functionality preservation.

In [ ]:
class CodeValidator:
    """Validates optimized code for syntax and basic functionality."""

    @staticmethod
    def validate_syntax(code: str) -> Tuple[bool, Optional[str]]:
        """Check if code has valid Python syntax."""
        try:
            ast.parse(code)
            return True, None
        except SyntaxError as e:
            return False, str(e)

    @staticmethod
    def basic_functionality_check(original: str, optimized: str) -> Dict[str, Any]:
        """Basic check to ensure function signatures are preserved."""
        try:
            orig_tree = ast.parse(original)
            opt_tree = ast.parse(optimized)

            orig_functions = [node.name for node in ast.walk(orig_tree)
                            if isinstance(node, ast.FunctionDef)]
            opt_functions = [node.name for node in ast.walk(opt_tree)
                           if isinstance(node, ast.FunctionDef)]

            orig_classes = [node.name for node in ast.walk(orig_tree)
                          if isinstance(node, ast.ClassDef)]
            opt_classes = [node.name for node in ast.walk(opt_tree)
                         if isinstance(node, ast.ClassDef)]

            return {
                "functions_preserved": set(orig_functions).issubset(set(opt_functions)),
                "classes_preserved": set(orig_classes).issubset(set(opt_classes)),
                "original_functions": orig_functions,
                "optimized_functions": opt_functions,
                "original_classes": orig_classes,
                "optimized_classes": opt_classes
            }
        except Exception as e:
            return {"error": str(e)}

    def validate_and_report(self, original: str, optimized: str) -> Dict[str, Any]:
        """Comprehensive validation with detailed reporting."""
        # Syntax validation
        syntax_valid, syntax_error = self.validate_syntax(optimized)
        
        # Functionality check
        functionality_check = self.basic_functionality_check(original, optimized)
        
        validation_result = {
            "syntax_valid": syntax_valid,
            "syntax_error": syntax_error,
            "functionality_check": functionality_check,
            "overall_valid": syntax_valid and functionality_check.get("functions_preserved", False)
        }
        
        return validation_result

    def display_validation_results(self, results: Dict[str, Any]):
        """Display validation results in a user-friendly format."""
        print("🔍 Code Validation Results")
        print("=" * 25)
        
        # Syntax validation
        if results["syntax_valid"]:
            print("✅ Syntax: Valid")
        else:
            print(f"❌ Syntax Error: {results['syntax_error']}")
        
        # Functionality check
        func_check = results["functionality_check"]
        if "error" not in func_check:
            if func_check.get("functions_preserved", False):
                print("✅ Functions: Preserved")
            else:
                print("⚠️ Functions: Some may be missing")
                print(f"   Original: {func_check.get('original_functions', [])}")
                print(f"   Optimized: {func_check.get('optimized_functions', [])}")
        else:
            print(f"❌ Functionality Check Error: {func_check['error']}")
        
        # Overall result
        if results["overall_valid"]:
            print("\n🎉 Overall: Code optimization is valid!")
        else:
            print("\n⚠️ Overall: Code optimization needs review")

# Test the validator
validator = CodeValidator()
print("✅ CodeValidator defined with comprehensive validation capabilities")

# Test with sample code
test_original = "def test_func(x): return x + 1"
test_optimized = "def test_func(x): return x + 1  # Same function"
results = validator.validate_and_report(test_original, test_optimized)
validator.display_validation_results(results)

## Main Optimization Framework {#main-framework}

The core framework that orchestrates the entire optimization process.

In [ ]:
class CodeOptimizationFramework:
    """Main framework orchestrating the optimization process."""

    def __init__(self, groq_api_key: str, model_name: str = "meta-llama/llama-4-maverick-17b-128e-instruct"):
        self.groq_client = Groq(api_key=groq_api_key)
        self.model_name = model_name
        self.validator = CodeValidator()

        # Initialize agents
        self.agents = {
            OptimizationType.PERFORMANCE: PerformanceOptimizationAgent(self.groq_client, model_name),
            OptimizationType.READABILITY: ReadabilityOptimizationAgent(self.groq_client, model_name),
            OptimizationType.MEMORY: MemoryOptimizationAgent(self.groq_client, model_name),
            OptimizationType.GENERAL: GeneralOptimizationAgent(self.groq_client, model_name)
        }

    def _check_event_loop(self):
        """Check if there's already a running event loop."""
        try:
            loop = asyncio.get_running_loop()
            return True
        except RuntimeError:
            return False

    def select_optimization_type(self, query: str) -> OptimizationType:
        """Automatically select optimization type based on query."""
        query_lower = query.lower()

        if any(word in query_lower for word in ['speed', 'fast', 'performance', 'optimize', 'efficient']):
            return OptimizationType.PERFORMANCE
        elif any(word in query_lower for word in ['readable', 'clean', 'understand', 'maintainable']):
            return OptimizationType.READABILITY
        elif any(word in query_lower for word in ['memory', 'ram', 'space', 'storage']):
            return OptimizationType.MEMORY
        else:
            return OptimizationType.GENERAL

    def _calculate_improvement_score(self, initial: Dict, current: Dict,
                                   initial_smells: int, current_smells: int) -> float:
        """Calculate improvement score based on various metrics."""
        score = 0.0

        # Complexity improvement
        if 'cyclomatic_complexity' in initial and 'cyclomatic_complexity' in current:
            if current['cyclomatic_complexity'] <= initial['cyclomatic_complexity']:
                score += 0.3

        # Code smells reduction
        if current_smells < initial_smells:
            score += 0.3 * (initial_smells - current_smells) / max(initial_smells, 1)

        # Lines of code (prefer concise but not at expense of readability)
        if 'lines_of_code' in initial and 'lines_of_code' in current:
            if current['lines_of_code'] <= initial['lines_of_code'] * 1.2:  # Allow 20% increase
                score += 0.2

        # Base score for valid syntax and no errors
        if 'error' not in current:
            score += 0.2

        return min(score, 1.0)

print("✅ CodeOptimizationFramework (Part 1) defined")

In [ ]:
# Continue CodeOptimizationFramework - Part 2 (Main optimization methods)

async def optimize_code_async(self, request: OptimizationRequest) -> OptimizationResult:
    """Main asynchronous optimization method."""
    logger.info(f"Starting optimization: {request.optimization_type.value}")

    # Get the appropriate agent
    agent = self.agents.get(request.optimization_type, self.agents[OptimizationType.GENERAL])

    # Initial analysis
    initial_analysis = agent.analyzer.analyze_complexity(request.code)
    original_smells = agent.analyzer.find_code_smells(request.code)

    best_code = request.code
    best_modifications = []
    confidence_score = 0.0
    iteration = 0

    for iteration in range(request.max_iterations):
        logger.info(f"Optimization iteration {iteration + 1}")

        # Create request for this iteration
        iter_request = OptimizationRequest(
            code=best_code,
            query=request.query,
            optimization_type=request.optimization_type,
            constraints=request.constraints
        )

        # Get optimization from agent
        optimized_code, modifications = await agent.optimize(iter_request)

        # Validate the optimized code
        is_valid, syntax_error = self.validator.validate_syntax(optimized_code)

        if not is_valid:
            logger.warning(f"Syntax error in iteration {iteration + 1}: {syntax_error}")
            continue

        # Check if this is better than previous iterations
        current_analysis = agent.analyzer.analyze_complexity(optimized_code)
        current_smells = agent.analyzer.find_code_smells(optimized_code)

        # Calculate improvement score
        improvement_score = self._calculate_improvement_score(
            initial_analysis, current_analysis, len(original_smells), len(current_smells)
        )

        if improvement_score > confidence_score:
            best_code = optimized_code
            confidence_score = improvement_score
            best_modifications = modifications

        # Early stopping if confidence is high enough
        if confidence_score > 0.8:
            break

    # Final analysis
    final_analysis = agent.analyzer.analyze_complexity(best_code)
    functionality_check = self.validator.basic_functionality_check(request.code, best_code)

    return OptimizationResult(
        original_code=request.code,
        optimized_code=best_code,
        modifications=best_modifications,
        analysis={
            "initial": initial_analysis,
            "final": final_analysis,
            "functionality_preserved": functionality_check,
            "code_smells": {
                "original": original_smells,
                "final": agent.analyzer.find_code_smells(best_code)
            }
        },
        confidence_score=confidence_score,
        iteration_count=iteration + 1,
        optimization_type=request.optimization_type.value,
        timestamp=datetime.now().isoformat()
    )

def optimize_code_sync(self, request: OptimizationRequest) -> OptimizationResult:
    """Synchronous optimization method for Jupyter compatibility."""
    logger.info(f"Starting synchronous optimization: {request.optimization_type.value}")

    # Get the appropriate agent
    agent = self.agents.get(request.optimization_type, self.agents[OptimizationType.GENERAL])

    # Initial analysis
    initial_analysis = agent.analyzer.analyze_complexity(request.code)
    original_smells = agent.analyzer.find_code_smells(request.code)

    best_code = request.code
    best_modifications = []
    confidence_score = 0.0
    iteration = 0

    for iteration in range(request.max_iterations):
        logger.info(f"Optimization iteration {iteration + 1}")

        # Create request for this iteration
        iter_request = OptimizationRequest(
            code=best_code,
            query=request.query,
            optimization_type=request.optimization_type,
            constraints=request.constraints
        )

        # Get optimization from agent (synchronous)
        prompt = agent.get_optimization_prompt(iter_request)

        try:
            response = agent.groq_client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model=agent.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            response_content = response.choices[0].message.content
            optimized_code = agent.extract_code_from_response(response_content)
            modifications = agent.extract_modifications_from_response(response_content, request.code, optimized_code)
        except Exception as e:
            logger.error(f"Groq API error: {e}")
            continue

        # Validate the optimized code
        is_valid, syntax_error = self.validator.validate_syntax(optimized_code)

        if not is_valid:
            logger.warning(f"Syntax error in iteration {iteration + 1}: {syntax_error}")
            continue

        # Check if this is better than previous iterations
        current_analysis = agent.analyzer.analyze_complexity(optimized_code)
        current_smells = agent.analyzer.find_code_smells(optimized_code)

        # Calculate improvement score
        improvement_score = self._calculate_improvement_score(
            initial_analysis, current_analysis, len(original_smells), len(current_smells)
        )

        if improvement_score > confidence_score:
            best_code = optimized_code
            confidence_score = improvement_score
            best_modifications = modifications

        # Early stopping if confidence is high enough
        if confidence_score > 0.8:
            break

    # Final analysis
    final_analysis = agent.analyzer.analyze_complexity(best_code)
    functionality_check = self.validator.basic_functionality_check(request.code, best_code)

    return OptimizationResult(
        original_code=request.code,
        optimized_code=best_code,
        modifications=best_modifications,
        analysis={
            "initial": initial_analysis,
            "final": final_analysis,
            "functionality_preserved": functionality_check,
            "code_smells": {
                "original": original_smells,
                "final": agent.analyzer.find_code_smells(best_code)
            }
        },
        confidence_score=confidence_score,
        iteration_count=iteration + 1,
        optimization_type=request.optimization_type.value,
        timestamp=datetime.now().isoformat()
    )

# Add methods to the framework class
CodeOptimizationFramework.optimize_code_async = optimize_code_async
CodeOptimizationFramework.optimize_code_sync = optimize_code_sync

print("✅ CodeOptimizationFramework (Part 2) optimization methods added")

In [ ]:
# Continue CodeOptimizationFramework - Part 3 (Convenience methods)

def quick_optimize(self, code: str, query: str, optimization_type: Optional[OptimizationType] = None) -> OptimizationResult:
    """Quick optimization with automatic type selection and sync execution."""
    if optimization_type is None:
        optimization_type = self.select_optimization_type(query)

    request = OptimizationRequest(
        code=code,
        query=query,
        optimization_type=optimization_type,
        max_iterations=2
    )

    return self.optimize_code_sync(request)

def get_optimization_json(self, code: str, query: str, pretty: bool = True) -> str:
    """Get optimization results as JSON string."""
    result = self.quick_optimize(code, query)
    return result.to_json(pretty=pretty)

def optimize_with_analysis(self, code: str, query: str, show_analysis: bool = True) -> OptimizationResult:
    """Optimize code with optional analysis display."""
    if show_analysis:
        print("📊 Original Code Analysis")
        print("=" * 30)
        analyzer = CodeAnalyzer()
        analyzer.analyze_and_display(code, "Pre-optimization")
        print("\n" + "="*50 + "\n")

    result = self.quick_optimize(code, query)

    if show_analysis:
        print("🎯 Optimization Results")
        print("=" * 25)
        result.display_summary()
        
        print("\n🔍 Validation Results")
        print("=" * 22)
        validation = self.validator.validate_and_report(result.original_code, result.optimized_code)
        self.validator.display_validation_results(validation)

    return result

def save_optimization_to_file(self, code: str, query: str, filename: str = "optimization_result.json") -> Optional[str]:
    """Save optimization results to a JSON file."""
    try:
        result = self.quick_optimize(code, query)
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(result.to_json())
        
        print(f"💾 Optimization results saved to {filename}")
        print(f"📊 Confidence: {result.confidence_score:.2%}")
        print(f"🔧 Modifications: {len(result.modifications)}")
        return result.to_json()
    
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

def compare_optimizations(self, code: str, query: str) -> Dict[str, OptimizationResult]:
    """Compare different optimization types for the same code."""
    optimization_types = [OptimizationType.PERFORMANCE, OptimizationType.READABILITY, 
                         OptimizationType.MEMORY, OptimizationType.GENERAL]
    
    results = {}
    
    print("🔄 Running comparison across all optimization types...\n")
    
    for opt_type in optimization_types:
        print(f"🎯 Testing {opt_type.value} optimization...")
        try:
            result = self.quick_optimize(code, query, opt_type)
            results[opt_type.value] = result
            print(f"   ✅ Confidence: {result.confidence_score:.2%}, Modifications: {len(result.modifications)}")
        except Exception as e:
            print(f"   ❌ Failed: {e}")
            
    return results

# Add methods to the framework class
CodeOptimizationFramework.quick_optimize = quick_optimize
CodeOptimizationFramework.get_optimization_json = get_optimization_json
CodeOptimizationFramework.optimize_with_analysis = optimize_with_analysis
CodeOptimizationFramework.save_optimization_to_file = save_optimization_to_file
CodeOptimizationFramework.compare_optimizations = compare_optimizations

print("✅ CodeOptimizationFramework (Part 3) convenience methods added")
print("🎉 Complete CodeOptimizationFramework is ready!")

## Usage Examples {#usage-examples}

Practical examples demonstrating the framework's capabilities.

### Initialize the Framework

In [ ]:
# Initialize the optimization framework
if GROQ_API_KEY != "your-groq-api-key-here":
    framework = CodeOptimizationFramework(GROQ_API_KEY, MODEL_NAME)
    print("✅ Framework initialized successfully!")
    print(f"🤖 Available optimization types: {[agent.value for agent in OptimizationType]}")
else:
    print("❌ Please set your GROQ_API_KEY in the configuration cell above")
    print("   You can get one from: https://console.groq.com/keys")

### Example 1: Basic Performance Optimization

In [ ]:
# Sample inefficient code for performance optimization
inefficient_code = """
def calculate_sum(numbers):
    total = 0
    for i in range(len(numbers)):
        total = total + numbers[i]
    return total

def find_max(numbers):
    max_val = numbers[0]
    for i in range(1, len(numbers)):
        if numbers[i] > max_val:
            max_val = numbers[i]
    return max_val

def process_data(data):
    results = []
    for item in data:
        if item > 0:
            results.append(item * 2)
    return results
"""

if 'framework' in locals():
    print("🚀 Optimizing for performance...\n")
    result = framework.optimize_with_analysis(
        inefficient_code, 
        "optimize this code for better performance",
        show_analysis=True
    )
    
    print("\n" + "="*60)
    print("📝 OPTIMIZED CODE:")
    print("="*60)
    print(result.optimized_code)
else:
    print("❌ Framework not initialized. Please run the initialization cell first.")

### Example 2: Readability Optimization

In [ ]:
# Code with poor readability
messy_code = """
def f(x,y,z):
    a=x+y
    b=a*z
    if b>100:
        return b-50
    elif b>50:
        return b*0.8
    else:
        return b+10

def proc(lst):
    r=[]
    for i in lst:
        if i%2==0:
            r.append(i*3)
    return r
"""

if 'framework' in locals():
    print("📖 Optimizing for readability...\n")
    result = framework.quick_optimize(
        messy_code, 
        "make this code more readable and maintainable"
    )
    
    result.display_summary()
    
    print("\n📝 ORIGINAL CODE:")
    print("-" * 40)
    print(messy_code)
    
    print("\n📝 OPTIMIZED CODE:")
    print("-" * 40)
    print(result.optimized_code)
else:
    print("❌ Framework not initialized. Please run the initialization cell first.")

### Example 3: Memory Optimization

In [ ]:
# Memory-intensive code
memory_heavy_code = """
def process_large_dataset(data):
    # Create multiple large lists
    squared = [x**2 for x in data]
    filtered = [x for x in squared if x > 100]
    doubled = [x*2 for x in filtered]
    final = [str(x) for x in doubled]
    return final

def generate_report(numbers):
    all_data = []
    for num in numbers:
        temp_list = [num * i for i in range(1000)]
        all_data.extend(temp_list)
    return all_data
"""

if 'framework' in locals():
    print("🧠 Optimizing for memory efficiency...\n")
    result = framework.quick_optimize(
        memory_heavy_code, 
        "optimize this code to use less memory"
    )
    
    result.display_summary()
    
    print("\n📝 OPTIMIZED CODE:")
    print("-" * 40)
    print(result.optimized_code)
    
    print("\n💡 Key Improvements:")
    for i, mod in enumerate(result.modifications, 1):
        print(f"   {i}. [{mod.type}] {mod.description}")
else:
    print("❌ Framework not initialized. Please run the initialization cell first.")

### Example 4: Automatic Optimization Type Selection

In [ ]:
# Test automatic optimization type selection
sample_code = """
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

def factorial(n):
    result = 1
    for i in range(1, n + 1):
        result = result * i
    return result
"""

if 'framework' in locals():
    # Test different queries to see auto-selection
    test_queries = [
        "make this code faster and more efficient",
        "improve readability and add documentation", 
        "reduce memory usage",
        "optimize this code"
    ]
    
    print("🔍 Testing automatic optimization type selection...\n")
    
    for query in test_queries:
        selected_type = framework.select_optimization_type(query)
        print(f"Query: '{query}'")
        print(f"   → Selected type: {selected_type.value}")
        print()
    
    # Demonstrate quick optimization with auto-selection
    print("\n🚀 Running optimization with auto-selection...")
    result = framework.quick_optimize(
        sample_code, 
        "make this recursive fibonacci more efficient"
    )
    
    print(f"\n🎯 Auto-selected optimization type: {result.optimization_type}")
    result.display_summary()
else:
    print("❌ Framework not initialized. Please run the initialization cell first.")

### Example 5: Compare All Optimization Types

In [ ]:
# Compare different optimization approaches
comparison_code = """
def process_text(text):
    words = text.split()
    result = []
    for word in words:
        if len(word) > 3:
            clean_word = ""
            for char in word:
                if char.isalpha():
                    clean_word += char.lower()
            if clean_word:
                result.append(clean_word)
    return result
"""

if 'framework' in locals():
    print("⚖️ Comparing all optimization types...\n")
    
    results = framework.compare_optimizations(
        comparison_code,
        "optimize this text processing function"
    )
    
    print("\n📊 COMPARISON SUMMARY:")
    print("=" * 50)
    
    for opt_type, result in results.items():
        print(f"\n🎯 {opt_type.upper()} Optimization:")
        print(f"   Confidence: {result.confidence_score:.2%}")
        print(f"   Modifications: {len(result.modifications)}")
        print(f"   Lines changed: {result._calculate_lines_changed()}")
        
        if result.modifications:
            print("   Key improvements:")
            for mod in result.modifications[:2]:  # Show top 2
                print(f"     • {mod.description[:60]}..." if len(mod.description) > 60 else f"     • {mod.description}")
    
    # Find the best optimization
    best_optimization = max(results.items(), key=lambda x: x[1].confidence_score)
    print(f"\n🏆 Best optimization: {best_optimization[0]} (confidence: {best_optimization[1].confidence_score:.2%})")
    
else:
    print("❌ Framework not initialized. Please run the initialization cell first.")

### Example 6: Save Results to File

In [ ]:
# Save optimization results to JSON file
save_example_code = """
def bubble_sort(arr):
    n = len(arr)
    for i in range(n):
        for j in range(0, n-i-1):
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return arr

def linear_search(arr, target):
    for i in range(len(arr)):
        if arr[i] == target:
            return i
    return -1
"""

if 'framework' in locals():
    print("💾 Saving optimization results to file...\n")
    
    json_result = framework.save_optimization_to_file(
        save_example_code,
        "optimize these algorithms for better performance",
        "algorithm_optimization.json"
    )
    
    if json_result:
        print("\n📄 File saved successfully! Here's a preview of the JSON structure:")
        
        # Show a preview of the JSON structure
        import json
        data = json.loads(json_result)
        
        print("\n🔍 JSON Structure Preview:")
        print(f"   • Optimization type: {data['optimization_summary']['type']}")
        print(f"   • Confidence score: {data['optimization_summary']['confidence_score']}")
        print(f"   • Total modifications: {data['optimization_summary']['total_modifications']}")
        print(f"   • Lines changed: {data['code']['lines_changed']}")
        print(f"   • Size change: {data['code']['size_change']['difference']} lines")
        
        if data['modifications']:
            print("   • Sample modifications:")
            for mod in data['modifications'][:2]:
                print(f"     - [{mod['type']}] {mod['description'][:50]}...")
else:
    print("❌ Framework not initialized. Please run the initialization cell first.")

## Advanced Features {#advanced-features}

Explore advanced capabilities and customization options.

### Custom Optimization Request

In [ ]:
# Create a custom optimization request with specific constraints
advanced_code = """
class DataProcessor:
    def __init__(self):
        self.data = []
        self.cache = {}
    
    def add_data(self, item):
        self.data.append(item)
    
    def process_all(self):
        results = []
        for item in self.data:
            if item in self.cache:
                result = self.cache[item]
            else:
                result = self.expensive_operation(item)
                self.cache[item] = result
            results.append(result)
        return results
    
    def expensive_operation(self, item):
        # Simulate expensive computation
        total = 0
        for i in range(1000):
            total += item * i
        return total
"""

if 'framework' in locals():
    print("🔧 Creating custom optimization request...\n")
    
    # Create a custom request
    custom_request = OptimizationRequest(
        code=advanced_code,
        query="optimize this class for both performance and maintainability",
        optimization_type=OptimizationType.GENERAL,
        constraints={
            "preserve_class_structure": True,
            "maintain_caching": True,
            "max_complexity_increase": 0.2
        },
        max_iterations=3
    )
    
    print(f"🎯 Optimization type: {custom_request.optimization_type.value}")
    print(f"🔄 Max iterations: {custom_request.max_iterations}")
    print(f"⚙️ Constraints: {custom_request.constraints}")
    
    # Run the optimization
    result = framework.optimize_code_sync(custom_request)
    
    print("\n📊 Custom Optimization Results:")
    result.display_summary()
    
    # Show the optimized code
    print("\n📝 OPTIMIZED CODE:")
    print("-" * 50)
    print(result.optimized_code)
    
else:
    print("❌ Framework not initialized. Please run the initialization cell first.")

### Detailed Analysis and Metrics

In [ ]:
# Detailed analysis of optimization results
if 'framework' in locals() and 'result' in locals():
    print("📊 DETAILED ANALYSIS")
    print("=" * 50)
    
    # Analysis data
    analysis = result.analysis
    
    print("\n🔍 COMPLEXITY ANALYSIS:")
    print("-" * 25)
    initial = analysis.get('initial', {})
    final = analysis.get('final', {})
    
    metrics = [
        ('Cyclomatic Complexity', 'cyclomatic_complexity'),
        ('Lines of Code', 'lines_of_code'),
        ('Function Count', 'function_count'),
        ('Class Count', 'class_count')
    ]
    
    for metric_name, key in metrics:
        initial_val = initial.get(key, 'N/A')
        final_val = final.get(key, 'N/A')
        
        if isinstance(initial_val, (int, float)) and isinstance(final_val, (int, float)):
            change = final_val - initial_val
            change_pct = (change / initial_val * 100) if initial_val != 0 else 0
            change_str = f"{change:+.1f} ({change_pct:+.1f}%)"
            emoji = "📈" if change > 0 else "📉" if change < 0 else "➡️"
        else:
            change_str = "N/A"
            emoji = "❓"
        
        print(f"{emoji} {metric_name}: {initial_val} → {final_val} ({change_str})")
    
    print("\n🚨 CODE SMELLS ANALYSIS:")
    print("-" * 25)
    smells = analysis.get('code_smells', {})
    original_smells = smells.get('original', [])
    final_smells = smells.get('final', [])
    
    print(f"Original code smells: {len(original_smells)}")
    print(f"Final code smells: {len(final_smells)}")
    
    smell_reduction = len(original_smells) - len(final_smells)
    if smell_reduction > 0:
        print(f"✅ Reduced by {smell_reduction} code smell(s)")
    elif smell_reduction < 0:
        print(f"⚠️ Increased by {abs(smell_reduction)} code smell(s)")
    else:
        print("➡️ No change in code smells")
    
    print("\n🔧 FUNCTIONALITY PRESERVATION:")
    print("-" * 30)
    func_check = analysis.get('functionality_preserved', {})
    
    if 'error' not in func_check:
        functions_ok = func_check.get('functions_preserved', False)
        classes_ok = func_check.get('classes_preserved', False)
        
        print(f"✅ Functions preserved: {functions_ok}" if functions_ok else f"⚠️ Functions preserved: {functions_ok}")
        print(f"✅ Classes preserved: {classes_ok}" if classes_ok else f"⚠️ Classes preserved: {classes_ok}")
        
        orig_funcs = func_check.get('original_functions', [])
        opt_funcs = func_check.get('optimized_functions', [])
        
        if orig_funcs:
            print(f"Original functions: {orig_funcs}")
            print(f"Optimized functions: {opt_funcs}")
    else:
        print(f"❌ Functionality check error: {func_check['error']}")
    
    print("\n📈 OPTIMIZATION SUMMARY:")
    print("-" * 25)
    print(f"🎯 Type: {result.optimization_type}")
    print(f"📊 Confidence: {result.confidence_score:.2%}")
    print(f"🔄 Iterations: {result.iteration_count}")
    print(f"📅 Timestamp: {result.timestamp}")
    print(f"📏 Lines changed: {result._calculate_lines_changed()}")
    
else:
    print("❌ No optimization result available. Please run an optimization first.")

## Framework Features Summary

**🎯 Optimization Types:**
- **Performance** - Speed and algorithmic efficiency
- **Readability** - Code clarity and maintainability
- **Memory** - Memory usage optimization
- **General** - Overall code quality improvements

**🔧 Key Features:**
- **AI-Powered Analysis** - Uses Groq models for intelligent optimization
- **Automatic Type Selection** - Chooses optimization type based on query
- **Code Validation** - Ensures syntax and functionality preservation
- **Iterative Improvement** - Multiple optimization iterations for better results
- **Comprehensive Metrics** - Detailed analysis and scoring
- **JSON Export** - Complete results in structured format
- **Comparison Tools** - Compare different optimization approaches

**📊 Analysis Capabilities:**
- Cyclomatic complexity calculation
- Code smell detection
- Function and class preservation checking
- Performance impact assessment
- Line-by-line change tracking

**🚀 Usage Patterns:**
- Quick optimization with auto-type selection
- Custom optimization requests with constraints
- Batch comparison across optimization types
- File-based result storage and analysis
- Interactive analysis and validation

This framework provides a comprehensive solution for AI-powered code optimization with detailed analysis, validation, and reporting capabilities!